These notebooks are part of Kaggle’s [Practical Model Evaluation](https://www.kaggle.com/practical-model-evaluation) event, which ran from December 3-5 2019. You can find the [livestreams for this event here](https://youtu.be/7RdKnACscjA?list=PLqFaTIg4myu-HA1VGJi_7IGFkKRoZeOFt).

* Day 1 Notebook: [Figuring out what matters for you](https://www.kaggle.com/rtatman/practical-model-evaluation-day-1)
* Day 2 Notebook: [Training models with automated machine learning](https://www.kaggle.com/rtatman/practical-model-evaluation-day-2)
* Day 3 Notebook: [Evaluating our models](https://www.kaggle.com/rtatman/practical-model-evaluation-day-3)

***

For today's exercise, we're going to be working on classifying roles into job titles based on information about the role. The data will be from the [2018](https://www.kaggle.com/kaggle/kaggle-survey-2018) and [2019](https://www.kaggle.com/c/kaggle-survey-2019) Kaggle data science survey. 

I've already [done some data cleaning](https://www.kaggle.com/rebeccaturner/data-prep-for-job-title-classification) but if you'd like to do your own or do some additional feature engineering, feel free!

Today we'll be building four different models using four different libraries, including some automated machine learning libraries. 

> Automated machine learning (or AutoML for short) is the task of removing human labor from the process of training machine learning models. Currently most AutoML research is focused on automating model selection and hyperparameter tuning. [This video](https://www.youtube.com/watch?v=Rsg_XzgGqZw&utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event) goes into more details.

The libraries we'll be using are:

* [XGBoost](https://xgboost.readthedocs.io/en/latest/) (not automated machine learning: we'll be using this as a baseline)
* [Cloud AutoML](https://cloud.google.com/automl/?utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event), an enterprise-focused automated machine learning product
* [TPOT](https://epistasislab.github.io/tpot/), an open source automated machine learning library developed at the University of Pennsylvania
* [H20.ai AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html), a second open source automated machine learning library developed by researchers at H20.ai

Let's get started!

## Load in data

First let's load in our pre-cleaned data. I'll be using the 2018 data as an example and then have you work through the 2019 data as your exercise.

In [1]:
# Importing libraries
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix
import pandas as pd
import category_encoders as ce

# set a seed for reproducability
random.seed(42)

# read in our data
df_2018 = pd.read_csv("../input/data-prep-for-job-title-classification/data_jobs_info_2018.csv")
df_2019 = pd.read_csv("../input/data-prep-for-job-title-classification/data_jobs_info_2019.csv")

In [2]:
df_2018.head()

,analyze_and_understand_data_to_influence_product_or_business_decisions,build_a_machine_learning_service_that_operationally_improves_my_product_or_workflows,build_the_data_infrastructure_that_my_business_uses_for_storing_analyzing_and_operationalizing_data,build_prototypes_to_explore_applying_machine_learning_to_new_areas,do_research_that_advances_the_state_of_the_art_of_machine_learning,none_of_these_activities_are_an_important_part_of_my_role_at_work,what_is_the_primary_tool_that_you_use_at_work_or_school_to_analyze_data,google_cloud_platform_gcp,amazon_web_services_aws,microsoft_azure,...,time_series_data,video_data,highest_level_of_formal_education,job_title,current_industry,years_of_experience,yearly_compensation,does_your_current_employer_incorporate_machine_learning_methods_into_their_business,which_ml_library_have_you_used_the_most_selected_choice,what_percent_of_your_time_at_work_is_spent_actively_coding
0,Analyze and understand data to influence produ...,Build and/or run a machine learning service th...,Build and/or run the data infrastructure that ...,NaN,Do research that advances the state of the art...,NaN,"Cloud-based data software & APIs (AWS, GCP, Az...",NaN,NaN,Microsoft Azure,...,NaN,NaN,Doctoral degree,Consultant,Other,NaN,NaN,I do not know,NaN,0% of my time
1,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,Local or hosted development environments (RStu...,NaN,NaN,NaN,...,Time Series Data,NaN,Master’s degree,Data Scientist,I am a student,0-1,"0-10,000",I do not know,NaN,75% to 99% of my time
2,NaN,NaN,NaN,NaN,NaN,NaN,"Advanced statistical software (SPSS, SAS, etc.)",NaN,NaN,Microsoft Azure,...,NaN,NaN,Master’s degree,Data Analyst,I am a student,0-1,"0-10,000",I do not know,NaN,75% to 99% of my time
3,Analyze and understand data to influence produ...,NaN,NaN,Build prototypes to explore applying machine l...,Do research that advances the state of the art...,NaN,Local or hosted development environments (RStu...,Google Cloud Platform (GCP),Amazon Web Services (AWS),NaN,...,Time Series Data,NaN,Bachelor’s degree,Data Scientist,Computers/Technology,0-1,I do not wish to disclose my approximate yearl...,We are exploring ML methods (and may one day p...,Keras,50% to 74% of my time
4,NaN,Build and/or run a machine learning service th...,Build and/or run the data infrastructure that ...,NaN,NaN,NaN,Local or hosted development environments (RStu...,NaN,Amazon Web Services (AWS),NaN,...,Time Series Data,NaN,Master’s degree,Software Engineer,Online Service/Internet-based Services,3-4,"20-30,000","We have well established ML methods (i.e., mod...",Scikit-Learn,75% to 99% of my time


In [3]:
df_2019.head()

,analyze_and_understand_data_to_influence_product_or_business_decisions,build_the_data_infrastructure_that_my_business_uses_for_storing_analyzing_and_operationalizing_data,build_prototypes_to_explore_applying_machine_learning_to_new_areas,build_a_machine_learning_service_that_operationally_improves_my_product_or_workflows,experimentation_and_iteration_to_improve_existing_ml_models,do_research_that_advances_the_state_of_the_art_of_machine_learning,none_of_these_activities_are_an_important_part_of_my_role_at_work,what_is_the_primary_tool_that_you_use_at_work_or_school_to_analyze_data,highest_level_of_formal_education,job_title,yearly_compensation,does_your_current_employer_incorporate_machine_learning_methods_into_their_business
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G...",Master’s degree,Software Engineer,"30,000-39,999",I do not know
1,Analyze and understand data to influence produ...,Build and/or run the data infrastructure that ...,Build prototypes to explore applying machine l...,Build and/or run a machine learning service th...,NaN,NaN,NaN,"Cloud-based data software & APIs (AWS, GCP, Az...",Professional degree,Software Engineer,"5,000-7,499","We have well established ML methods (i.e., mod..."
2,NaN,NaN,Build prototypes to explore applying machine l...,NaN,NaN,Do research that advances the state of the art...,NaN,"Advanced statistical software (SPSS, SAS, etc.)",Master’s degree,Data Scientist,"60,000-69,999","We have well established ML methods (i.e., mod..."
3,Analyze and understand data to influence produ...,NaN,NaN,NaN,Experimentation and iteration to improve exist...,Do research that advances the state of the art...,NaN,"Local development environments (RStudio, Jupyt...",Master’s degree,Data Scientist,"10,000-14,999",We are exploring ML methods (and may one day p...
4,Analyze and understand data to influence produ...,NaN,Build prototypes to explore applying machine l...,Build and/or run a machine learning service th...,NaN,NaN,NaN,"Local development environments (RStudio, Jupyt...",Bachelor’s degree,Data Scientist,"80,000-89,999","We recently started using ML methods (i.e., mo..."


## Data preparation

We do have an additional step of preperation. First, we'll split into training and testing sets:

In [4]:
# split into predictor & target variables
X = df_2018.drop("job_title", axis=1)
y = df_2018["job_title"]

# Splitting data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20)

# save out the split training data to use with Cloud AutoML
with open("train_data_2018.csv", "+w") as file:
    pd.concat([X_train, y_train], axis=1).to_csv(file, index=False)
with open("test_data_2018.csv", "+w") as file:
    pd.concat([X_test, y_test], axis=1).to_csv(file, index=False)

For H20 AutoML and Cloud AutoML we don't need to do anything else. (Actually for Cloud AutoML we don't even need to split our data, but we'll look at that in a minute.) 

For TPOT and XGBoost, however, we need to make sure that all our input data is numeric. We'll be using [ordinal label encoding](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) for this.

In [5]:
# encode all features using ordinal encoding
encoder_x = ce.OrdinalEncoder()
X_encoded = encoder_x.fit_transform(X)

# you'll need to use a different encoder for each dataframe
encoder_y = ce.OrdinalEncoder()
y_encoded = encoder_y.fit_transform(y)

# split encoded dataset
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded, y_encoded,
                                                    train_size=0.80, test_size=0.20)

# XGBoost Baseline

First, we're going to train a basic XGBoost model using the default arguments. We cover XGBoost in more detail in the [Intro To Machine Learning course](https://www.kaggle.com/learn/intro-to-machine-learning), so I'm not going to talk about it here.

In [6]:
from xgboost import XGBClassifier

# train XGBoost model with default parameters
my_model = XGBClassifier()
my_model.fit(X_train_encoded, y_train_encoded, verbose=False)

# and save our model
my_model.save_model("xgboost_baseline.model")

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
my_model.predict(X_test_encoded)

array([4, 2, 2, ..., 2, 2, 2])

# Cloud AutoML

Now let's train our Cloud AutoML model! We'll be using both the GCP console and notebook code here, so you'll probably want to open those in separate tabs or windows.

### Prepare your account and project


* First you’ll need to [create a GCP account](https://accounts.google.com/signup/v2/webcreateaccount?service=cloudconsole&continue=https%3A%2F%2Fconsole.cloud.google.com%2F&dsh=S-385463669%3A1575309184770524&gmb=exp&biz=false&flowName=GlifWebSignIn&flowEntry=SignUp&nogm=true&utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event) (if you already have a Google account you can use that one) and [enable billing](https://www.youtube.com/watch?v=uINleRduCWM&utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event).


> For now, you do need to have a credit card in order to enable billing and you need billing enabled to use Cloud AutoML. If you’re not able to enable billing you can still follow along with the rest of the workshop, just skip the Cloud AutoML parts. 

 

* From there, [create a new project](https://cloud.google.com/appengine/docs/standard/nodejs/building-app/creating-project?utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event). You should [set the region of your project](https://cloud.google.com/compute/docs/regions-zones/?utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event) to “us-central1”.

* Go to the [AutoML Tables](https://console.cloud.google.com/automl-tables?utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event) page in the Google Cloud Console and click *enable API*. This will let you train an AutoML Tables model in your current project. 


### Creating your dataset


For this workshop, we’re going to create our AutoML datasets using the GCP console. The reason for this is that importing datasets can take a while. If you have the code in your notebook to import your dataset right before the code to create your model, when you run your notebook top to bottom it’ll give you an error because the modelling code was run before the dataset was done importing.


* Click on “Datasets” in the list on the left hand side of your screen and then click on the blue **[+] New Dataset** text near the top of your screen.

* Give your dataset a name and make sure the region is **US-CENTRAL1**.  

* Select “Upload files from your computer” and select the file with the dataset you want. 

* Click on **BROWSE** under the “Select Files” button and a side panel will pop up. 

    * If you haven’t created any buckets you’ll see the text “No buckets found”. To create a new bucket, click on the icon that looks like a shopping basket with a plus sign in it.

    * Follow the prompts to create your bucket. **Important:** Make sure in the “Choose where to store your data” step, you pick “Region” and set the location as “us-central1 (Iowa). 

* Select the bucket where you’d like to store your data.

* Import your dataset. (This may take a while.)

* Once your dataset is done importing, take a close look at your imported data and make sure it looks the way you’d expect.


### Training our model


In order to train an AutoML model from inside Kaggle Notebooks, you’ll need to attach a notebook to your Google Cloud Account. [This video goes into more detail](https://youtu.be/xP99eh6nQN0?utm_medium=notebook&utm_source=kaggle&utm_campaign=automl-event). 


Then you can modify the following code to start your AutoML model training:

In [8]:
from google.cloud import automl_v1beta1 as automl
from kaggle.gcp import KaggleKernelCredentials
from kaggle_secrets import GcpTarget
from google.cloud import storage

# don't change this value!
REGION = 'us-central1' # don't change: this is the only region that works currently

# these you'll change based on your GCP project/data
PROJECT_ID = 'gm-kaggle-1' # this will come from your specific GCP project
DATASET_DISPLAY_NAME = 'dataset_job_2018_1575641833179' # name of your uploaded dataset (from GCP console)
TARGET_COLUMN = 'job_title' # column with feature you're trying to predict

# these can be whatever you like
MODEL_DISPLAY_NAME = 'kaggle_automl_example_model1' # what you want to call your model
TRAIN_BUDGET = 1000 # max time to train model in milli-hours, from 1000-72000

storage_client = storage.Client(project=PROJECT_ID, credentials=KaggleKernelCredentials(GcpTarget.GCS)) 
tables_gcs_client = automl.GcsClient(client=storage_client, credentials=KaggleKernelCredentials(GcpTarget.GCS)) 
tables_client = automl.TablesClient(project=PROJECT_ID, region=REGION, gcs_client=tables_gcs_client, credentials=KaggleKernelCredentials(GcpTarget.AUTOML))

In [9]:
# first you'll need to make sure your model is predicting the right column
tables_client.set_target_column(
    dataset_display_name=DATASET_DISPLAY_NAME,
    column_spec_display_name=TARGET_COLUMN,
)

name: "projects/203368842829/locations/us-central1/datasets/TBL8951920208142925824"
display_name: "dataset_job_2018_1575641833179"
create_time {
  seconds: 1575641967
  nanos: 606150000
}
etag: "AB3BwFrI6B3z5FzG6Ho_PSl4SgwhYbZS23eIPj7JjcK1pt0AQ-4cH_OxqdigVQTzmro="
example_count: 13862
tables_dataset_metadata {
  primary_table_spec_id: "8306722386917457920"
  target_column_spec_id: "731362149447761920"
  target_column_correlations {
    key: "1019592525599473664"
    value {
      cramers_v: 0.01589312144662912
    }
  }
  target_column_correlations {
    key: "1307822901751185408"
    value {
      cramers_v: 0.026228333569284504
    }
  }
  target_column_correlations {
    key: "154901397144338432"
    value {
      cramers_v: 0.010363774712942754
    }
  }
  target_column_correlations {
    key: "1596053277902897152"
    value {
      cramers_v: 0.03745949678439566
    }
  }
  target_column_correlations {
    key: "2172514030206320640"
    value {
      cramers_v: 0.02634104952670167

In [10]:
# let our model know that input columns may have missing values
for col in tables_client.list_column_specs(project=PROJECT_ID,
                                           dataset_display_name=DATASET_DISPLAY_NAME):
    if TARGET_COLUMN in col.display_name:
        continue
    tables_client.update_column_spec(project=PROJECT_ID,
                                     dataset_display_name=DATASET_DISPLAY_NAME,
                                     column_spec_display_name=col.display_name,
                                     nullable=True)

In [11]:
# and then you'll need to kick off your model training
response = tables_client.create_model(MODEL_DISPLAY_NAME, dataset_display_name=DATASET_DISPLAY_NAME, 
                                      train_budget_milli_node_hours=TRAIN_BUDGET, 
                                      exclude_column_spec_names=[TARGET_COLUMN])

# check if it's done yet (it won't be)
response.done()

False

Once our model starts training, we don't need to do anything else: it's already saved in our GCP account and good to go for tomorrow.

# TPOT

Alright, now we'll move onto [TPOT](https://epistasislab.github.io/tpot/). This is an academic library built on top of scikit-learn, and my favorite thing about it is that when you export a model you're actually exporting all the Python code you need to train that model.

In [12]:
from tpot import TPOTClassifier

# create & fit TPOT classifier with 
tpot = TPOTClassifier(generations=8, population_size=20, 
                      verbosity=2, early_stop=2)
tpot.fit(X_train_encoded, y_train_encoded)

# save our model code
tpot.export('tpot_pipeline.py')

# print the model code to see what it says
!cat tpot_pipeline.py

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Generation 1 - Current best internal CV score: 0.4935508049607237

The optimized pipeline was not improved after evaluating 2 more generations. Will end the optimization process.

TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.1, max_depth=10, min_child_weight=16, n_estimators=100, nthread=1, subsample=0.3)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=None)

# Average CV score on the training set was:0.4935508049607237
exported_pipeline = XGBClassifier(learning_rate=0.1, max_dep

# H20.ai AutoML

For our final model we'll be using [H20.ai's open source AutoML library](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html). One thing that I like about this library is that, as each model is trained, its evaluated both on its own and as part of a stacked ensemble. Kaggle competitors are very fond of stacking (and H20 is known for hiring a lot of top Kagglers) so it's nice to have that automated for us.

In [13]:
import h2o
from h2o.automl import H2OAutoML

# initilaize an H20 instance running locally
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-1~deb9u1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9ui_h8_c
  JVM stdout: /tmp/tmp9ui_h8_c/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp9ui_h8_c/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,1 month and 19 days
H2O cluster name:,H2O_from_python_unknownUser_zf7t8t
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [14]:
# convert our data to h20Frame, an alternative to pandas datatables
train_data = h2o.H2OFrame(X_train)
test_data = h2o.H2OFrame(list(y_train))

train_data = train_data.cbind(test_data)

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(y="C1", training_frame=train_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [15]:
# View the top five models from the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=5)

# The leader model can be access with `aml.leader`

model_id,mean_per_class_error,logloss,rmse,mse
GLM_grid_1_AutoML_20191207_112852_model_1,0.678643,1.4374,0.695061,0.48311
XGBoost_1_AutoML_20191207_112852,0.682858,1.44783,0.700149,0.490208
GBM_5_AutoML_20191207_112852,0.684204,1.47622,0.695915,0.484297
XGBoost_2_AutoML_20191207_112852,0.686176,1.45283,0.702352,0.493299
GBM_1_AutoML_20191207_112852,0.688342,1.49487,0.696846,0.485595


In [16]:
# save the model out (we'll need to for tomorrow!)
h2o.save_model(aml.leader)

'/kaggle/working/GLM_grid_1_AutoML_20191207_112852_model_1'

# Check that we've saved each of our models

Before we wrap up for the day, we want to make sure we've saved all of our models for tomorrow! The Cloud AutoML model is saved automatically on GCP, but we've saved each of the other models in our current working directory. Let's just double check that that's the case:

In [17]:
# check to see that we've saved all of our models
! ls

GLM_grid_1_AutoML_20191207_112852_model_1  tpot_pipeline.py
__notebook__.ipynb			   train_data_2018.csv
__output__.json				   xgboost_baseline.model
test_data_2018.csv


Alright, we've got three models and the code for the notebook. We're all set!

# Exercise

Now it's your turn! Following the steps above, use the `df_2019` dataframe and: 

* Prepare your data (split into testing and training, encode variables)
* Train your models: XGBoost, Cloud AutoML, TPOT and H20 AutoML

> Note: if you can't or would prefer not to set up billing on order to use Cloud AutoML, feel free to skip training that model.

* Remember to save your models! You'll need them tomorrow and, since it takes a while to run AutoML code, you don't want to have to run it multiple times.

Have fun training your models and I'll see you all tomorrow for our final model evaluation!